In [1]:
library(GUniFrac)

### Simulation

In [2]:
file_paths <- c("~/from_pendrive/PhD/DAA_benchmark_study/qiime_ancombc/data/ecam_10245/ecam_otu_table.tsv",
                "~/from_pendrive/PhD/DAA_benchmark_study/qiime_ancombc/data/PD_10483/pd_10483_otu_table.tsv")

In [9]:
list_of_dfs <- lapply(file_paths, function(file) read.table(file, sep = '\t', header=TRUE, row.names=1))


In [11]:
names(list_of_dfs) <- c("ecam", "pd")


In [33]:
#sample_size=list(length(rownames(list_of_dfs$ecam)),length(rownames(list_of_dfs$pd)))
sample_size=list(1000,1200)

In [30]:
sample_size[[2]]

[1] 1249

In [28]:
nsample=list(length(list_of_dfs$ecam),length(list_of_dfs$pd))

In [31]:
nsample[[2]]

[1] 410

In [34]:
num_dfs <- length(list_of_dfs)
num_sizes <- length(sample_size)
simulation <- vector("list", num_dfs)

for(i in 1:length(list_of_dfs)) {
    simulation[[i]] <- vector("list", num_sizes)
    for(j in 1:length(sample_size)){
    simulation[[i]][[j]] <- SimulateMSeq(
ref.otu.tab = list_of_dfs[[i]], nSam = 200, nOTU = sample_size[[j]],
# True signal setting
diff.otu.pct = 0.1, diff.otu.direct = c("unbalanced"), 
diff.otu.mode = c("abundant"),
covariate.type = c("binary"), grp.ratio = 1,
covariate.eff.mean = 1.0, covariate.eff.sd = 0,
# Confounder signal setting
confounder.type = c("both"), conf.cov.cor = 0.6,
conf.diff.otu.pct = 0.1, conf.nondiff.otu.pct = 0.1,
confounder.eff.mean = 1.0, confounder.eff.sd = 0,
# Depth setting
depth.mu = 10000, depth.theta = 5, depth.conf.factor = 0
)
 }   
}


Iteration 1: Log-likelihood value: -12958604.7384067

Iteration 2: Log-likelihood value: -12936632.967925

Iteration 3: Log-likelihood value: -12916488.3009465

Iteration 4: Log-likelihood value: -12898298.5036623

Iteration 5: Log-likelihood value: -12882182.0803318

Iteration 6: Log-likelihood value: -12868353.5488289

Iteration 7: Log-likelihood value: -12857134.5282301

Iteration 8: Log-likelihood value: -12848860.6662821

Iteration 9: Log-likelihood value: -12843624.477581

Iteration 10: Log-likelihood value: -12841004.4186788

Iteration 11: Log-likelihood value: -12840101.0531563

Iteration 12: Log-likelihood value: -12839934.2027679

Iteration 13: Log-likelihood value: -12839924.0579665

Iteration 14: Log-likelihood value: -12839923.9869446

Iteration 15: Log-likelihood value: -12839923.986939

Iteration 1: Log-likelihood value: -12958604.7384067

Iteration 2: Log-likelihood value: -12936632.967925

Iteration 3: Log-likelihood value: -12916488.3009465

Iteration 4: Log-likelihoo

### Method run

In [43]:
run_method<- function(raw_data, DA_method){

  if(DA_method=="ANCOM-BC"){
    out<- run_ancombc(raw_data = raw_data)
  } else if(DA_method=="FlashWeave"){
    out<- run_flashweave(raw_data = raw_data)
  
  }

  return(out)
}


run_flashweave<- function(raw_data){

    setwd("/home/zakir/from_pendrive/PhD/DAA_benchmark_study/qiime2_benchmark")
    # data file paths
    original_data<-"~/from_pendrive/PhD/DAA_benchmark_study/qiime_ancombc/data/PD_10483/otu_pd_qiime.tsv"
    simulated_data<-"/home/zakir/from_pendrive/PhD/DAA_benchmark_study/qiime2_benchmark/otu_table1.txt"
    biom_file <- '/home/zakir/from_pendrive/PhD/DAA_benchmark_study/qiime2_benchmark/otu_table1.biom'
    output_qza <- '/home/zakir/from_pendrive/PhD/DAA_benchmark_study/qiime2_benchmark/otu_table1.qza'
    metadata<-"/home/zakir/from_pendrive/PhD/DAA_benchmark_study/qiime2_benchmark/meta_table1.tsv"
    output_flash <- '/home/zakir/from_pendrive/PhD/DAA_benchmark_study/qiime2_benchmark/flashnet1.qza'
    neighbour='/home/zakir/from_pendrive/PhD/DAA_benchmark_study/qiime2_benchmark/neighbours1.qza'
    exported <- '/home/zakir/from_pendrive/PhD/DAA_benchmark_study/qiime2_benchmark/exported_neighbours'
    
    meta.dat <- data.frame(X = raw_data$covariate, Z1 = raw_data$confounder[, 1],
                       Z2 = raw_data$confounder[, 2])
    otu.tab.sim <- raw_data$otu.tab.sim
    actual=ifelse(raw_data$diff.otu.ind, "DA", "NOT_DA")
    tr=data.frame("truth" = actual, row.names = rownames(otu.tab.sim), stringsAsFactors = F)
     meta.dat$X=ifelse(meta.dat$X, "healthy","unhealthy")
    #otu data processing for qiime2 input
    otu <- cbind(rownames(otu.tab.sim), otu.tab.sim[,2:length(colnames(otu.tab.sim))])
    colnames(otu)[1] <- "featureid"
    rownames(otu) <- NULL

    write.table(otu, simulated_data, sep = "\t", col.names=TRUE, row.names=FALSE, quote = FALSE)
    #metadata processing for qiime2 input
    colnames(meta.dat)=c("covariate", "cofounder1","cofounder2")

    metad  <- cbind(rownames(meta.dat),meta.dat)
    colnames(metad)[1] <- "sampleid"
    rownames(metad) <- NULL
    write.table(metad, metadata, sep = "\t", col.names=TRUE, row.names=FALSE, quote = FALSE)

   # convert data into biom format
    system(paste("biom convert",
                 "-i",simulated_data ,
                 "-o", biom_file ,
                 "--table-type=\"OTU table\"",
                 "--to-hdf5"), intern=TRUE)

    #import data into qiime 
    system(paste("qiime tools import",
             "--input-path", biom_file,
             "--type 'FeatureTable[Frequency]'",
             "--input-format BIOMV210Format",
             "--output-path", output_qza), intern=TRUE)
  
    # Run FlashWeave with sampled parameters
    system(paste("qiime makarsa flashweave",
             "--i-table", output_qza,
             "--m-metadata-file",metadata,
             "--p-heterogeneous", TRUE,         #need to replace parameters with their best 
             "--p-sensitive", TRUE,             #value found in hyperparameter tuning.
             "--p-max-k", 0,
             "--p-alpha", 0.05 ,
             "--o-network", output_flash), intern=TRUE)                              
  
 
 
    system(paste("qiime makarsa list-neighbours",
                "--i-network",output_flash,
                "--p-feature-id", "covariate",
                "--o-neighbours",neighbour), intern=TRUE)                 

   # Export the results from QIIME2
   system(paste("qiime tools export", 
              "--input-path",neighbour,
             "--output-path",exported), intern=TRUE)                
  
  
  
  # Construct file path for metadata.tsv
   metadata_tsv <- paste0(exported, "/metadata.tsv")
  
  # Check if the file exists before reading
  if (!file.exists(metadata_tsv)) {
    cat("Expected metadata.tsv does not exist at path:", metadata_tsv, "\n")
    next
  }
  
  # Read the results
  neighbours <-read.csv(file = metadata_tsv, sep = '\t', row.names = 1)
  
  # take only necessary data
    neighbours=  neighbours[-1,] 
    e=rownames(otu.tab.sim)%in%rownames(neighbours)   
    pred=data.frame("pred" =ifelse(e, "DA", "NOT_DA") , row.names =rownames(otu.tab.sim) , stringsAsFactors = F)
    out_df=cbind(pred,tr) 
    
  # Return a list with the output of the method and the running time:
  return("method_out"=out_df)
}



run_ancombc<- function(raw_data){

    
    setwd("/home/zakir/from_pendrive/PhD/DAA_benchmark_study/qiime2_benchmark")
    # data file paths
    original_data<-"~/from_pendrive/PhD/DAA_benchmark_study/qiime_ancombc/data/PD_10483/otu_pd_qiime.tsv"
    simulated_data<-"/home/zakir/from_pendrive/PhD/DAA_benchmark_study/qiime2_benchmark/otu_table1.txt"
    biom_file <- '/home/zakir/from_pendrive/PhD/DAA_benchmark_study/qiime2_benchmark/otu_table1.biom'
    output_qza <- '/home/zakir/from_pendrive/PhD/DAA_benchmark_study/qiime2_benchmark/otu_table1.qza'
    metadata<-"/home/zakir/from_pendrive/PhD/DAA_benchmark_study/qiime2_benchmark/meta_table1.tsv"
    ancomoutput <- '/home/zakir/from_pendrive/PhD/DAA_benchmark_study/qiime2_benchmark/ancombc.qza'
    exported <- '/home/zakir/from_pendrive/PhD/DAA_benchmark_study/qiime2_benchmark/exported_ancompd'
    
    meta.dat <- data.frame(X = raw_data$covariate, Z1 = raw_data$confounder[, 1],
                       Z2 = raw_data$confounder[, 2])
    otu.tab.sim <- raw_data$otu.tab.sim
    actual=ifelse(raw_data$diff.otu.ind, "DA", "NOT_DA")
    tr=data.frame("truth" = actual, row.names = rownames(otu.tab.sim), stringsAsFactors = F)
    meta.dat$X=ifelse(meta.dat$X, "healthy","unhealthy")
    #otu data processing for qiime2 input
    otu <- cbind(rownames(otu.tab.sim), otu.tab.sim[,2:length(colnames(otu.tab.sim))])
    colnames(otu)[1] <- "featureid"
    rownames(otu) <- NULL

    write.table(otu, simulated_data, sep = "\t", col.names=TRUE, row.names=FALSE, quote = FALSE)
    #metadata processing for qiime2 input
    colnames(meta.dat)=c("covariate", "cofounder1","cofounder2")

    metad  <- cbind(rownames(meta.dat),meta.dat)
    colnames(metad)[1] <- "sampleid"
    rownames(metad) <- NULL
    write.table(metad, metadata, sep = "\t", col.names=TRUE, row.names=FALSE, quote = FALSE)

   # convert data into biom format
    system(paste("biom convert",
                 "-i",simulated_data ,
                 "-o", biom_file ,
                 "--table-type=\"OTU table\"",
                 "--to-hdf5"), intern=TRUE)
    #import data into qiime2                
    system(paste("qiime tools import",
             "--input-path", biom_file,
             "--type 'FeatureTable[Frequency]'",
             "--input-format BIOMV210Format",
             "--output-path", output_qza), intern=TRUE)
                   
   # Run ANCOM-BC with sampled parameters
    system(paste("qiime composition ancombc",
             "--i-table",output_qza,
             "--m-metadata-file", metadata,
             "--p-formula", "covariate",
             "--p-p-adj-method", "holm",
            # "--p-zero-cut", 0.7,
             "--p-lib-cut", 50,
            # "--p-struct-zero", FALSE,
             "--p-neg-lb", FALSE,
             "--p-tol", 1e-04,
             "--p-max-iter", 50,
             "--p-conserve", TRUE,
             "--p-alpha", 0.03,
            # "--p-global", TRUE,
             "--o-differentials", ancomoutput))                
  
   # Export the results from QIIME2
   system(paste("qiime tools export", 
              "--input-path",ancomoutput,
             "--output-path",exported))                
  
  # Construct file path for q_val_slice.csv
   q_val_csv <- paste0(exported, "/q_val_slice.csv")
  
  # Check if the file exists before reading
  if (!file.exists(q_val_csv)) {
    cat("Expected q_val_slice.csv does not exist at path:", q_val_csv, "\n")
    next
  }
  
    # Read the q-value results
    ancom_q_val <- read.csv(file = q_val_csv, sep = ',', row.names = 1)

    # take only necessary data
    ancom_q_val=data.frame("covariateunhealthy" = ancom_q_val$covariateunhealthy, row.names = rownames(ancom_q_val), stringsAsFactors = F)
    #construct output dataframe
    e=!(rownames(otu.tab.sim)%in%rownames(ancom_q_val))
    nq=rownames(otu.tab.sim)[e]
    nqv=data.frame("covariateunhealthy" = rep(NA, length(nq)), row.names = nq, stringsAsFactors = F)
    q_val_all=rbind(ancom_q_val,nqv)
    names(q_val_all)[names(q_val_all) == "covariateunhealthy"] <- "q_value"
    pred= ifelse(ancom_q_val < 0.05, "DA", "NOT_DA")
    pred=as.data.frame(pred)
    d=!(rownames(otu.tab.sim)%in%rownames(pred))
    npred=rownames(otu.tab.sim)[d]
    ND=data.frame("covariateunhealthy" = rep("NOT_DA", length(npred)), row.names = npred, stringsAsFactors = F)
    pr=rbind(pred,ND)
    out_df=cbind(pr,q_val_all,tr)
    names(out_df)[names(out_df) == "covariateunhealthy"] <- "pred"

  return("method_out"=out_df)
}



### Running methods on simulated data

In [44]:
# List of methods you want to run
methods_to_run <- c("ANCOM-BC", "FlashWeave")  # Add more if needed

# Initialize results_list
results_list <- list()
for(method in methods_to_run) {
  results_list[[method]] <- vector("list", length(list_of_dfs))
  for(i in 1:length(list_of_dfs)) {
    results_list[[method]][[i]] <- vector("list", length(sample_size))
  }
}

# Loop over datasets and methods
for(i in 1:length(list_of_dfs)) {
  for(j in 1:length(sample_size)) {
    data <- simulation[[i]][[j]]
    
    for(method in methods_to_run) {
      # Run the method
      result <- run_method(data, method)
      
      # Store the result
      results_list[[method]][[i]][[j]] <- result
    }
  }
}


In [46]:
#results_list

### RUNNING OUTPUT TO GENERATE PR-RECALL DATAFRAME

In [49]:
# Compute Precision and Recall
# param:
#   - truth: a vector that contains the labels that identify the simulated otu as "DA" or "NOT_DA".
#   - pred: a vector that contains the predicted labels.
# return: a named vector with precision and recall

compute_precision_recall<- function(truth, pred){

  if(all(is.na(pred))){
    out<- c(NA,NA)
    names(out)<- c("recall", "precision")
    return(out)
  }

  # Reassign the labels. The positive class is the one with the lowest number, i.e. "DA":
  truth<- ifelse(truth == "DA", 1, 0)
  pred<- ifelse(pred == "DA", 1, 0)

  # Compute the number of False Positive (FP), False Negative (FN), True Positive (TP), True Negative (TN):
  FP<- sum(truth==0 & pred==1)
  FN<- sum(truth==1 & pred==0)
  TP<- sum(truth==1 & pred==1)
  TN<- sum(truth==0 & pred==0)

  # Calculate Precision and Recall:
  if(TP==0 & FP==0){
    precision<- NA
  }else{
    precision<- TP/(TP+FP)
  }

  if(TP==0 & FN==0){
    recall<- NA
  }else{
    recall<- TP/(TP+FN)
  }

  if(FP==0 & TN==0){
    fpr<- NA
  }else{
    fpr<- FP/(FP+TN)
  }

  f1_score<- 2*(recall*precision)/(precision+recall)

  # Return precision and recall:
  out<- c(recall,precision,fpr, f1_score)
  names(out)<- c("recall", "precision","fpr", "f1_score")
  return(out)
}


In [50]:
# Initialize a dataframe to store precision-recall metrics
precision_recall_df <- data.frame()

for(method in methods_to_run) {
    recall_vector <- c()
    precision_vector <- c()
    fpr_vector <- c()
    f1_vector <- c()
    
    for(i in 1:length(list_of_dfs)) {
        for(j in 1:length(sample_size)) {
            method_results <- results_list[[method]][[i]][[j]]
            
            prec_rec_out <- compute_precision_recall(truth = method_results$truth, pred = method_results$pred)
            
            recall_vector <- c(recall_vector, prec_rec_out["recall"])
            precision_vector <- c(precision_vector, prec_rec_out["precision"])
            fpr_vector <- c(fpr_vector, prec_rec_out["fpr"])
            f1_vector <- c(f1_vector, prec_rec_out["f1_score"])
        }
    }
    
    tmp_df <- data.frame("method" = rep(method, length(recall_vector)),
                         "recall" = recall_vector,
                         "precision" = precision_vector,
                         "fpr" = fpr_vector,
                         "F1" = f1_vector,
                         "na_precision" = rep(sum(is.na(precision_vector)), length(recall_vector)),
                         stringsAsFactors = F)
    
    precision_recall_df <- rbind(precision_recall_df, tmp_df)
}


In [51]:
precision_recall_df

method,recall,precision,fpr,F1,na_precision
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
ANCOM-BC,0.04000000,0.5000000,0.0044444444,0.07407407,0
ANCOM-BC,0.05833333,0.7777778,0.0018518519,0.10852713,0
ANCOM-BC,0.03000000,0.3333333,0.0066666667,0.05504587,0
ANCOM-BC,0.04166667,0.4166667,0.0064814815,0.07575758,0
FlashWeave,0.05000000,0.3333333,0.0111111111,0.08695652,0
FlashWeave,0.03333333,0.8000000,0.0009259259,0.06400000,0
FlashWeave,0.18000000,0.5625000,0.0155555556,0.27272727,0
FlashWeave,0.10833333,0.7647059,0.0037037037,0.18978102,0
